In [1]:
#  This notebook will look over the weather data from multible folders and subfolders and extract point values

In [2]:
# import all the things 
from multiprocessing import Pool
from tqdm.auto import tqdm
from pathlib import Path
import datetime as dt
import os
import re
import rasterio
import geopandas as gpd

In [4]:
# point this at the weather data
input_folder = '/home/nick/Documents/Work code/Dam-Forecast/data'

# this is just a small sample use 'all dams centroids with water and dates.gpkg' for all dams
input_file = 'all dams centroids with water and dates sample.gpkg'
input_path = os.path.join(input_folder,input_file)
# make sure this exists
os.path.isfile(input_path)

True

In [5]:
# point this at your weatehr data folder
weather_folder = '/mnt/2TB Working/Projects/Dam forecast/Weather raster data'

In [6]:
# this is our output folder
time_step_folder = '/mnt/2TB Working/Projects/Dam forecast/v4/forecast/time steps'
# make this folder if it does not exist
Path(time_step_folder).mkdir(parents=True, exist_ok=True)

In [6]:
# depeding on how you got your weatehr data it may be in one of many raster formats, 
# this script works with both .nc and .adf files
# you can downlaod data from
# https://dapds00.nci.org.au/thredds/catalogs/gh70/v2-0/v2-0.html

# get a refence to all raster files
raster_list = []
for root, dirs, files in os.walk(weather_folder):
    for file in files:
        if file.endswith(('1.adf','.nc')):
            raster_list.append(os.path.join(root, file))
len(raster_list) 

927

In [7]:
# all raster files 
raster_list

['/mnt/2TB Working/Projects/Dam forecast/Weather raster data/VPD/y2010/vpd1008/w001001.adf',
 '/mnt/2TB Working/Projects/Dam forecast/Weather raster data/VPD/y2010/vpd1002/w001001.adf',
 '/mnt/2TB Working/Projects/Dam forecast/Weather raster data/VPD/y2010/vpd1010/w001001.adf',
 '/mnt/2TB Working/Projects/Dam forecast/Weather raster data/VPD/y2010/vpd2010annu/w001001.adf',
 '/mnt/2TB Working/Projects/Dam forecast/Weather raster data/VPD/y2010/vpd1006/w001001.adf',
 '/mnt/2TB Working/Projects/Dam forecast/Weather raster data/VPD/y2010/vpd1009/w001001.adf',
 '/mnt/2TB Working/Projects/Dam forecast/Weather raster data/VPD/y2010/vpd1004/w001001.adf',
 '/mnt/2TB Working/Projects/Dam forecast/Weather raster data/VPD/y2010/vpd1012/w001001.adf',
 '/mnt/2TB Working/Projects/Dam forecast/Weather raster data/VPD/y2010/vpd1001/w001001.adf',
 '/mnt/2TB Working/Projects/Dam forecast/Weather raster data/VPD/y2010/vpd1011/w001001.adf',
 '/mnt/2TB Working/Projects/Dam forecast/Weather raster data/VPD/y

In [8]:
# depedning on your data source the climate variable labels may be inconsistant
# the below variables are for rain and temp
include_types =['tavg','avgt','rain','Rain','Temp']

In [9]:
# limit raster list to only the varibles listed above
short_list=[]
for raster in raster_list:
    for inc_type in include_types:
        if inc_type in raster:
            short_list.append(raster)


In [10]:
print(len(short_list))
short_list

542


['/mnt/2TB Working/Projects/Dam forecast/Weather raster data/2019 - all rasters/rain_y2019/y2019/rain1902/w001001.adf',
 '/mnt/2TB Working/Projects/Dam forecast/Weather raster data/2019 - all rasters/rain_y2019/y2019/rain1908/w001001.adf',
 '/mnt/2TB Working/Projects/Dam forecast/Weather raster data/2019 - all rasters/rain_y2019/y2019/rain1905/w001001.adf',
 '/mnt/2TB Working/Projects/Dam forecast/Weather raster data/2019 - all rasters/rain_y2019/y2019/rain1910/w001001.adf',
 '/mnt/2TB Working/Projects/Dam forecast/Weather raster data/2019 - all rasters/rain_y2019/y2019/rain1912/w001001.adf',
 '/mnt/2TB Working/Projects/Dam forecast/Weather raster data/2019 - all rasters/rain_y2019/y2019/rain1901/w001001.adf',
 '/mnt/2TB Working/Projects/Dam forecast/Weather raster data/2019 - all rasters/rain_y2019/y2019/rain1911/w001001.adf',
 '/mnt/2TB Working/Projects/Dam forecast/Weather raster data/2019 - all rasters/rain_y2019/y2019/rain1909/w001001.adf',
 '/mnt/2TB Working/Projects/Dam forecast

In [11]:
# open vector data and reproject to WGS 84 if not already
points = gpd.read_file(input_path).to_crs("EPSG:4326")
points

,file_name,area,area_2,SRC_DATE,file_name_pred,class,class_name,geometry
0,arcgisonline_1686584.tif,1472.265188,293.852789,20160217.0,arcgisonline_1686584_pred_and_score.tif,0,dam and water,POINT (121.81364 -33.00995)
1,arcgisonline_1686963.tif,3511.531725,1348.727385,20150121.0,arcgisonline_1686963_pred_and_score.tif,0,dam and water,POINT (121.56878 -33.11647)
2,arcgisonline_437392.tif,7400.931025,NaN,20161230.0,arcgisonline_437392_pred_and_score.tif,2,no water,POINT (147.19717 -29.86993)
3,arcgisonline_1747284.tif,2925.907475,2924.952538,20160302.0,arcgisonline_1747284_pred_and_score.tif,1,no dam,POINT (147.48198 -42.99338)
4,arcgisonline_411890.tif,5270.397670,15.310323,20150315.0,arcgisonline_411890_pred_and_score.tif,2,no water,POINT (144.99258 -34.25549)
...,...,...,...,...,...,...,...,...
2995,arcgisonline_1504356.tif,2301.714042,1048.661742,20170131.0,arcgisonline_1504356_pred_and_score.tif,0,dam and water,POINT (141.36489 -37.11587)
2996,arcgisonline_1756404.tif,3196.282699,2112.732946,20150208.0,arcgisonline_1756404_pred_and_score.tif,0,dam and water,POINT (147.19517 -41.49876)
2997,arcgisonline_725535.tif,674.353129,NaN,20160326.0,arcgisonline_725535_pred_and_score.tif,2,no water,POINT (148.45037 -34.50040)
2998,arcgisonline_1304801.tif,2743.747761,NaN,20161111.0,arcgisonline_1304801_pred_and_score.tif,2,no water,POINT (144.58016 -37.89659)


In [12]:
# get list of years and months which will be the end dates for the time series
years = list(range(2011,2021))
print(years)
months = list(range(1,13))
print(months)

[2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


In [13]:
# how many years of data did you want to extract?
historic_years = 1
historic_months = historic_years*12
print(historic_months,'months')

12 months


In [14]:
# func to calc the number of months between two dates
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

In [15]:
# extracts metadata from nc files 
def nc_getter(raster):
    raster_name = os.path.basename(raster)
    raster_name_split = raster_name.split('_')
    raster_type = raster_name_split[2]
    if raster_type == 'tavg':
        raster_type = 'avgt'
    raster_year = int(raster_name_split[4][:4])
    raster_month = int(raster_name_split[4][4:6])
    return(raster_name,raster_type,raster_year,raster_month)

In [16]:
# extracts metadata from adf files
def adf_getter(raster):
    raster_name = raster.split('/')[-2]
    raster_type = re.search('(\D*)',raster_name)[0]
    raster_year = int('20'+re.search('(\d\d)',raster_name)[0])
    raster_month = int(re.search('(\d\d$)',raster_name)[0])
    return(raster_name,raster_type,raster_year,raster_month)

In [17]:
# func to extract point values from raster, it will also pull raster meta with the funcs above
def extract_raster_vals(raster):
# exclude in annu in path, this removed yearly totals
    if 'annu' not in raster: 
#         check for adf files
        if raster.endswith('.adf'):
            raster_name,raster_type,raster_year,raster_month = adf_getter(raster)
#         if not adf assume its nc
        else:
            raster_name,raster_type,raster_year,raster_month = nc_getter(raster)
#       build datetime object from raster date, we need to set a day so just pick 1st
        raster_date = dt.date(year=raster_year, month=raster_month, day=1)

        #checks if point values are more recent (or the same) as the raster
        if sample_date>=raster_date:

            #how many months between the raster and the sample
            month_dif = diff_month(sample_date,raster_date)
            
#           check month dif is within selected range
            if month_dif <= historic_months:
#               make column name from month dif (month offset)
                col_name = raster_type+'_'+str(month_dif)+'_'+'months before'
    
                #open raster
                src = rasterio.open(raster)
                raster_vals = []
#               pull out all point values at once
                for val in src.sample(sample_coord_list):
#                     print(val[0])
                    raster_vals.append(val[0])
                #return a dict of name and raster point values
                return {'name':col_name,'list':raster_vals}
        
#           if it failes at any the the steps aboce return an empty array 
            else:
                return []
        else:
            return []
    else:
        return []
        

In [18]:
# build list of each year and moth combo
start_times = []
for year in years:
    for month in months:
        start_times.append({'year' : year, 'month' : month})
len(start_times)

120

In [19]:
#grab a copy of the input data
sample_chunk = points
sample_coord_list = []
#       biuld list of coords to sample
print('setting up')
for index, row in tqdm(sample_chunk.iterrows(),total = sample_chunk.shape[0]):
    coord = (row['geometry'].x,row['geometry'].y)
    sample_coord_list.append(coord)


setting up


  0%|          | 0/3000 [00:00<?, ?it/s]

In [20]:
# loop over each year month combo
for time_step in tqdm(start_times):
#     extract each year and month 
    sample_year = time_step['year']
    sample_month = time_step['month']
    sample_date = dt.date(year=sample_year, month=sample_month, day=1)
#     build filename from start date
    file_name = str(sample_year)+'-'+str(sample_month)+'.csv'
    save_path = os.path.join(time_step_folder,file_name)

#     check if already done
    if not os.path.isfile(save_path):
#         use a multithreaded pool to extract data

        with Pool() as p:
            multi_cols = list(tqdm(p.imap(extract_raster_vals, short_list), 
                                   total=len(short_list),leave=False,
                                  desc='Month chunk '+str(sample_date)))

        for col in multi_cols:
            if type(col) == dict:
                sample_chunk[col['name']] = col['list']
#         export to disk
        sample_chunk.to_csv(save_path)


  0%|          | 0/120 [00:00<?, ?it/s]

Month chunk 2011-01-01:   0%|          | 0/542 [00:00<?, ?it/s]

Month chunk 2011-02-01:   0%|          | 0/542 [00:00<?, ?it/s]

Month chunk 2011-03-01:   0%|          | 0/542 [00:00<?, ?it/s]

Month chunk 2011-04-01:   0%|          | 0/542 [00:00<?, ?it/s]

...

Month chunk 2020-08-01:   0%|          | 0/542 [00:00<?, ?it/s]

Month chunk 2020-09-01:   0%|          | 0/542 [00:00<?, ?it/s]

Month chunk 2020-10-01:   0%|          | 0/542 [00:00<?, ?it/s]

Month chunk 2020-11-01:   0%|          | 0/542 [00:00<?, ?it/s]

Month chunk 2020-12-01:   0%|          | 0/542 [00:00<?, ?it/s]